Training a character level Transformer model built with pytorch on the first Harry Potter book.

In [ ]:
import torch
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Importin own modules and functions
from models import Transformer
from load_hp_data import load_data

Load the dataset.

In [7]:
data_file = 'hp_dataset/01 Harry Potter and the Sorcerers Stone.txt'
text, vocab_size, encode, decode = load_data(data_file)
data = torch.tensor(encode(text), dtype=torch.long, device=device)

# # Write the augmented text to a file
# with open('hp_dataset/augmented_hp_1.txt', 'w') as f:
#     f.write(decode(data.tolist()))

print(data.shape, data.dtype)

length of raw dataset in characters:  439478
Characters present in the raw dataset:  
 !'()*,-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz–—‘’“”…
Vocabulary size of the raw dataset:  82


length of dataset after augmentation in characters:  435847
Characters present in the augmented dataset:  
 !"',-.:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz…
Vocabulary size of the augmented dataset:  64


torch.Size([435847]) torch.int64


Example from the dataset.

In [8]:
print(text[:124])
print('\n')
print(data[:124])

M r. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.


tensor([23,  1, 54,  7,  1, 37, 50, 40,  1, 23, 54, 55,  7,  1, 14, 57, 54, 55,
        48, 41, 61,  5,  1, 51, 42,  1, 50, 57, 49, 38, 41, 54,  1, 42, 51, 57,
        54,  5,  1, 26, 54, 45, 58, 41, 56,  1, 14, 54, 45, 58, 41,  5,  1, 59,
        41, 54, 41,  1, 52, 54, 51, 57, 40,  1, 56, 51,  1, 55, 37, 61,  1, 56,
        44, 37, 56,  1, 56, 44, 41, 61,  1, 59, 41, 54, 41,  1, 52, 41, 54, 42,
        41, 39, 56, 48, 61,  1, 50, 51, 54, 49, 37, 48,  5,  1, 56, 44, 37, 50,
        47,  1, 61, 51, 57,  1, 58, 41, 54, 61,  1, 49, 57, 39, 44,  7],
       device='cuda:0')


Simple chronological split into train and validation sets.

In [9]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
def get_batch(split, batch_size, context_len):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_len, (batch_size,))
    x = torch.stack([data[i:i+context_len] for i in ix])
    y = torch.stack([data[i+1:i+context_len+1] for i in ix])
    return x, y

Initializing the model and the optimizer.

In [11]:
context_len = 256
embed_dim = 384
n_heads = 6
n_blocks = 6
batch_size = 64
dropout_rate = 0.2
learning_rate = 3e-4

max_iters = 5000
eval_interval = 250
eval_iters = 50

model = Transformer(vocab_size, context_len, embed_dim, n_heads, n_blocks, dropout_rate)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# Default div_factor 25 and final_div_factor 1000
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, total_steps=max_iters, pct_start=0.1, anneal_strategy='cos')

In [12]:
@torch.no_grad()
def estimate_loss():
    """ Helper function to estimate loss on train and val splits. """
    out = {}
    model.eval() # Set model to evaluation mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, batch_size, context_len)
            logits = model(X)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            Y = Y.view(B*T)
            loss = F.cross_entropy(logits, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # Set model back to training mode
    return out

Training loop.

In [ ]:
for iter_num in range(max_iters):

    if iter_num % eval_interval == 0 or iter_num == max_iters - 1:
        losses = estimate_loss()
        print(f"Step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Get a batch of data
    xb, yb = get_batch('train', batch_size, context_len)

    # Forward pass and loss calculation
    logits = model(xb)
    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    yb = yb.view(B*T)
    loss = F.cross_entropy(logits, yb)

    # Backward pass and optimization
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) 

    optimizer.step()
    scheduler.step()

print("\nTraining finished!")

Save the model.

In [ ]:
import os

SAVE_PATH = 'saved_models/model.pth'
os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)

torch.save(model.state_dict(), SAVE_PATH)